# Tugas 3: Menggunakan Amazon Bedrock untuk Menjawab Pertanyaan

Dalam <i>notebook</i> ini, Anda akan belajar cara menggunakan model Bedrock Titan untuk memberikan respons informasi terhadap pertanyaan dengan mengirimkan permintaan dengan konteks yang relevan penuh ke model dan mengharapkan respons balik, mengatasi tantangan dalam membuat model memberikan jawaban faktual untuk pertanyaan tanpa perlu menyiapkan dan mengindeks dokumen terlebih dahulu.

<i>Notebook</i> ini mensimulasikan apa yang akan dilakukan **<i>Retrieval-Augmented Generation</i> (RAG)**, tetapi tidak benar-benar menggunakan RAG. Pendekatan ini dapat diterapkan pada dokumen pendek atau aplikasi tunggal. Pendekatan ini mungkin tidak dapat diterapkan pada jawaban pertanyaan di tingkat korporasi, karena mungkin ada dokumen korporasi berukuran besar yang tidak semuanya dapat dimasukkan ke dalam promotor yang dikirim ke model.

**Menjawab Pertanyaan (QA)** adalah tugas penting yang melibatkan ekstraksi jawaban atas pertanyaan faktual yang diajukan dalam bahasa alami. Biasanya, sistem QA memproses kueri terhadap basis pengetahuan yang berisi data terstruktur atau tidak terstruktur dan menghasilkan respons dengan informasi yang akurat. Memastikan akurasi yang tinggi adalah kunci untuk mengembangkan sistem menjawab pertanyaan yang berguna, andal, dan dapat dipercaya, terutama untuk kasus penggunaan korporasi.


## Skenario

Anda mencoba memodelkan situasi di AnyCompany yang memungkinkan Anda mengajukan pertanyaan kepada model yang menjawab guna memberikan informasi tentang penggantian ban untuk model kendaraan tertentu yang mereka produksi. Pertama, Anda menanyakan model menggunakan pendekatan “<i>zero shot</i>” untuk melihat apakah model dapat memberikan jawaban yang relevan hanya berdasarkan data pelatihannya.

Namun, Anda menyadari model tersebut tampaknya "berhalusinasi" dengan jawaban yang lebih umum, terbukti ketika Anda mencoba model kendaraan palsu dan mendapatkan respons serupa. Hal ini menyiratkan perlunya melengkapi pelatihan model dengan manual kendaraan sebenarnya dari Example Company untuk memberikan informasi spesifik tentang ban untuk setiap model.

Di lab ini, Anda akan mensimulasikan pendekatan “<i>Retrieval Augmented Generation</i>” (RAG) tanpa data eksternal. Anda memberikan kutipan manual terperinci yang menjelaskan cara mengganti ban pada kendaraan AnyCompany Model Z. Anda menguji apakah model sekarang dapat memberikan jawaban yang disesuaikan dan akurat dengan memanfaatkan konten contoh dalam konteks ini.

## Tugas 3.1: Pengaturan lingkungan

Dalam tugas ini, Anda mengatur lingkungan Anda.

In [ ]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3
import botocore

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))



## Tugas 3.2: Tanya Jawab dengan pengetahuan model
Pada bagian ini kita mencoba menggunakan model yang disediakan oleh layanan Bedrock untuk menjawab pertanyaan berdasarkan pengetahuan yang diperoleh model selama fase pelatihan.

Dalam tugas ini, Anda menggunakan metode invoke_model () dari klien Amazon Bedrock. Parameter wajib yang diperlukan untuk menggunakan metode ini adalah modelId yang mewakili model Amazon Bedrock ARN, dan <i>body</i>, yang merupakan perintah untuk tugas Anda.

Perintah <i>body</i> berubah tergantung penyedia model fondasi yang Anda pilih. Anda dapat melihat penjelasannya secara detail di bawah.

```json
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```

Anda mencoba menggunakan model yang disediakan oleh layanan Bedrock untuk menjawab pertanyaan berdasarkan pengetahuan yang diperoleh selama fase pelatihan.

In [ ]:
prompt_data = """You are an helpful assistant. Answer questions in a concise way. If you are unsure about the
answer say 'I am unsure'

Question: How can I fix a flat tire on my AnyCompany AC8?
Answer:"""
parameters = {
    "maxTokenCount":512,
    "stopSequences":[],
    "temperature":0,
    "topP":0.9
    }

## Tugas 3.3: Menginvokasi model dengan meneruskan <i>body</i> JSON untuk menghasilkan respons

In [ ]:
#model configuration
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"
try:
    
    response = bedrock_client.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())
    answer = response_body.get("results")[0].get("outputText")
    print(answer.strip())

except botocore.exceptions.ClientError as error:
    if  error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Message']}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error


Model ini memberi Anda jawaban yang menguraikan proses penggantian ban kempes mobil, tetapi penjelasan yang sama bisa berlaku untuk mobil apa pun. Sayangnya, ini bukan jawaban yang tepat untuk AnyCompany AC8 yang tidak memiliki ban serep. Ini terjadi karena model telah dilatih dengan data yang berisi instruksi penggantian ban pada mobil.

Contoh lain masalah ini dapat dilihat dengan mencoba mengajukan pertanyaan yang sama untuk merek dan model mobil palsu, misalnya Amazon Tirana.

In [ ]:
prompt_data = "How can I fix a flat tire on my Amazon Tirana?"
body = json.dumps({"inputText": prompt_data, 
                   "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock_client.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print(answer.strip())

Mengingat perintah yang cepat, model tidak dapat memberikan jawaban yang realistis.

Untuk memperbaiki masalah ini dan meminta model memberikan jawaban berdasarkan instruksi khusus yang berlaku untuk model mobil Anda, Anda dapat menambah pengetahuan model secara langsung dengan memberikan basis pengetahuan tambahan sebagai bagian dari perintah.

Mari kita lihat bagaimana Anda dapat menggunakan ini untuk meningkatkan aplikasi.

Asumsikan ini adalah kutipan buku manual AnyCompany AC8 (sebenarnya ini bukan buku manual sebenarnya, tetapi anggap saja begitu). Dokumen ini juga cukup singkat sehingga sesuai dengan jendela konteks Titan Large.

```plain
Ban dan Tekanan Ban:

Ban terbuat dari karet berwarna hitam dan dipasang pada roda kendaraan Anda. Ban memberikan cengkeraman yang diperlukan untuk mengemudi, berbelok, dan mengerem. Dua faktor penting yang perlu diperhatikan adalah tekanan ban dan keausan ban, karena dapat memengaruhi kinerja dan penanganan mobil Anda.

Tempat untuk Menemukan Tekanan Ban yang Direkomendasikan:

Anda dapat menemukan spesifikasi tekanan ban yang direkomendasikan pada label pemompaan yang ada di pilar B sisi pengemudi kendaraan. Sebagai alternatif, Anda juga dapat melihat manual kendaraan Anda untuk informasi ini. Tekanan ban yang disarankan dapat bervariasi, tergantung pada kecepatan dan jumlah penumpang atau muatan maksimum di dalam kendaraan.

Mengisi Ulang Angin Ban:

Saat memeriksa tekanan ban, penting untuk melakukannya saat ban dalam keadaan dingin. Itu berarti mendiamkan kendaraan setidaknya selama tiga jam untuk memastikan suhu ban sama dengan suhu sekitar.

Cara mengisi ulang angin ban:

    Periksa tekanan ban yang direkomendasikan untuk kendaraan Anda.
    Ikuti instruksi yang diberikan pada pompa udara dan pompa ban hingga mencapai tekanan yang benar.
    Di tampilan tengah kendaraan Anda, buka aplikasi "Status mobil".
    Buka tab “Tekanan ban”.
    Tekan opsi “Kalibrasi tekanan” dan konfirmasi tindakan.
    Kendarai mobil selama beberapa menit dengan kecepatan di atas 30 km/jam untuk mengalibrasi tekanan ban.

Catatan: Dalam beberapa kasus, mungkin Anda perlu berkendara lebih dari 15 menit untuk menghapus simbol atau pesan peringatan apa pun yang terkait dengan tekanan ban. Jika peringatan terus muncul, dinginkan ban dan ulangi langkah di atas.

Ban Kempis:

Jika ban kempis saat mengemudi, Anda dapat menutup sementara sobekan pada ban tersebut dan mengisi kembali ban menggunakan alat mobilitas ban. Alat ini biasanya disimpan di bawah lapisan area bagasi kendaraan Anda.

Instruksi Penggunaan Alat Mobilitas Ban:

    Buka pintu belakang atau bagasi kendaraan Anda.
    Angkat lapisan area bagasi untuk mengakses alat mobilitas ban.
    Ikuti instruksi yang diberikan bersama alat mobilitas ban untuk menambal sobekan pada ban.
    Setelah menggunakan alat, pastikan untuk mengembalikannya dengan aman di lokasi asalnya.
    Hubungi Rivesla atau layanan yang sesuai untuk mendapatkan bantuan dalam membuang dan mengganti botol las ban bekas.

Harap dicatat bahwa alat mobilitas ban adalah solusi sementara dan dirancang untuk memungkinkan Anda berkendara maksimal 10 menit atau 8 km (mana saja yang lebih dulu) dengan kecepatan maksimum 80 km/jam. Disarankan untuk mengganti ban yang bocor atau membawanya untuk diperbaiki tenaga profesional sesegera mungkin.
```

In [ ]:
context = """Tires and tire pressure:

Tires are made of black rubber and are mounted on the wheels of your vehicle. They provide the necessary grip for driving, cornering, and braking. Two important factors to consider are tire pressure and tire wear, as they can affect the performance and handling of your car.

Where to find recommended tire pressure:

You can find the recommended tire pressure specifications on the inflation label located on the driver's side B-pillar of your vehicle. Alternatively, you can refer to your vehicle's manual for this information. The recommended tire pressure may vary depending on the speed and the number of occupants or maximum load in the vehicle.

Reinflating the tires:

When checking tire pressure, it is important to do so when the tires are cold. This means allowing the vehicle to sit for at least three hours to ensure the tires are at the same temperature as the ambient temperature.

To reinflate the tires:

    Check the recommended tire pressure for your vehicle.
    Follow the instructions provided on the air pump and inflate the tire(s) to the correct pressure.
    In the center display of your vehicle, open the "Car status" app.
    Navigate to the "Tire pressure" tab.
    Press the "Calibrate pressure" option and confirm the action.
    Drive the car for a few minutes at a speed above 30 km/h to calibrate the tire pressure.

Note: In some cases, it may be necessary to drive for more than 15 minutes to clear any warning symbols or messages related to tire pressure. If the warnings persist, allow the tires to cool down and repeat the above steps.

Flat Tire:

If you encounter a flat tire while driving, you can temporarily seal the puncture and reinflate the tire using a tire mobility kit. This kit is typically stored under the lining of the luggage area in your vehicle.

Instructions for using the tire mobility kit:

    Open the tailgate or trunk of your vehicle.
    Lift up the lining of the luggage area to access the tire mobility kit.
    Follow the instructions provided with the tire mobility kit to seal the puncture in the tire.
    After using the kit, make sure to securely put it back in its original location.
    Contact AnyCompany or an appropriate service for assistance with disposing of and replacing the used sealant bottle.

Please note that the tire mobility kit is a temporary solution and is designed to allow you to drive for a maximum of 10 minutes or 8 km (whichever comes first) at a maximum speed of 80 km/h. It is advisable to replace the punctured tire or have it repaired by a professional as soon as possible."""

##### Sekarang, berikan seluruh kutipannya ke model bersama dengan pertanyaan.

In [ ]:
question = "How can I fix a flat tire on my AnyCompany AC8?"
prompt_data = f"""Answer the question based only on the information provided between ## and give step by step guide.
#
{context}
#

Question: {question}
Answer:"""

### Tugas 3.4: Menginvokasi model melalui boto3 untuk menghasilkan respons

In [ ]:
body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = "amazon.titan-text-express-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock_client.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
answer = response_body.get("results")[0].get("outputText")
print(answer.strip())

Karena model membutuhkan waktu cukup lama untuk memahami konteks dan menghasilkan jawaban yang relevan untuk Anda, hal ini mungkin menyebabkan pengalaman buruk bagi pengguna karena mereka harus menunggu respons selama beberapa detik.

Bedrock juga mendukung kemampuan <i>streaming</i> tempat layanan menghasilkan <i>output</i> saat model menghasilkan token. Berikut contoh cara Anda bisa melakukannya.

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
# response with stream
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

Responsnya memberikan ringkasan petunjuk langkah demi langkah tentang cara mengganti ban. 

Anda sekarang telah mempelajari cara memanfaatkan <i>Retrieval Augmented Generation</i> (RAG) atau proses Augmentasi untuk menghasilkan respons yang dikurasi dan disesuaikan dengan konteks dan informasi spesifik yang diberikan.

### Cobalah sendiri
- Ubah perintah untuk kasus penggunaan spesifik Anda dan evaluasi <i>output</i> model yang berbeda.
- Atur panjang token untuk memahami latensi dan responsivitas layanan.
- Terapkan berbagai prinsip rekayasa perintah untuk mendapatkan <i>output</i> yang lebih baik.

### Pembersihan

Anda telah menyelesaikan <i>notebook</i> ini. Untuk pindah ke bagian lab berikutnya, lakukan hal berikut:

- Tutup <i>file notebook</i> ini dan lanjutkan dengan **Tugas 4**.